In [1]:
#Starting Saprksession for our assignment
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("spark-assignmentjulianb").getOrCreate()



Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/11/26 19:09:51 INFO SparkEnv: Registering MapOutputTracker
24/11/26 19:09:51 INFO SparkEnv: Registering BlockManagerMaster
24/11/26 19:09:51 INFO SparkEnv: Registering BlockManagerMasterHeartbeat
24/11/26 19:09:51 INFO SparkEnv: Registering OutputCommitCoordinator


In [4]:
#Loading the music dataset
path= "gs://sparkassignment4/dataset.csv"
file_type = "csv"

infer_schema = "true"
first_row_is_header = "true"
delimiter = ","


In [5]:
#Import Csv
df = spark.read.format(file_type)\
    .option("inferSchema",infer_schema)\
    .option("header",first_row_is_header)\
    .option("sep",delimiter)\
.load(path)

df.printSchema()
df.show(5)


root
 |-- _c0: integer (nullable = true)
 |-- track_id: string (nullable = true)
 |-- artists: string (nullable = true)
 |-- album_name: string (nullable = true)
 |-- track_name: string (nullable = true)
 |-- popularity: string (nullable = true)
 |-- duration_ms: string (nullable = true)
 |-- explicit: string (nullable = true)
 |-- danceability: string (nullable = true)
 |-- energy: string (nullable = true)
 |-- key: string (nullable = true)
 |-- loudness: string (nullable = true)
 |-- mode: string (nullable = true)
 |-- speechiness: string (nullable = true)
 |-- acousticness: string (nullable = true)
 |-- instrumentalness: double (nullable = true)
 |-- liveness: string (nullable = true)
 |-- valence: string (nullable = true)
 |-- tempo: double (nullable = true)
 |-- time_signature: double (nullable = true)
 |-- track_genre: string (nullable = true)

+---+--------------------+--------------------+--------------------+--------------------+----------+-----------+--------+------------+---

In [6]:
from pyspark.sql.functions import avg, desc

artists_avg_stats = df.groupBy("artists").agg(
    avg("danceability").alias("avg_danceability"),
    avg("energy").alias("avg_energy"),
    avg("tempo").alias("avg_tempo")
)

# Here we have the 5 artists with the highest average track popularity
top_artists_popularity = df.groupBy("artists").agg(
    avg("popularity").alias("avg_popularity")
).orderBy(desc("avg_popularity")).limit(5)

# Displaying the results
print("Average Danceability, Energy, and Tempo by Artist:")
artists_avg_stats.show()

print("Top 5 Artists with Highest Average Popularity:")
top_artists_popularity.show()


Average Danceability, Energy, and Tempo by Artist:


+--------------------+------------------+-------------------+------------------+
|             artists|  avg_danceability|         avg_energy|         avg_tempo|
+--------------------+------------------+-------------------+------------------+
|       Baobinga;I.D.|             0.666|              0.966|           136.991|
| Robert Owens;Atjazz|              NULL|               NULL|            0.0324|
|Johann Sebastian ...|            0.0769|             0.0846|           176.329|
|     Little Big Town|             0.459|0.48285714285714293|143.53457142857144|
|   Avicii;Aloe Blacc|             0.802|              0.645|           100.001|
|Lucas Estrada;Twa...|             0.614|              0.802|            98.008|
| ARMNHMR;Bella Renee|             0.405|              0.646|           141.985|
|      PEEKABOO;Eptic|             0.562|              0.934|           140.026|
|          BEAUZ;JVNA|             0.739|              0.796|           102.038|
| Tayrone;Leo Santana|      

+--------------------+--------------+
|             artists|avg_popularity|
+--------------------+--------------+
|Sam Smith;Kim Petras|         100.0|
|    Bizarrap;Quevedo|          99.0|
|       Manuel Turizo|          98.0|
|Bad Bunny;Chencho...|          97.0|
|Bad Bunny;Bomba E...|          94.5|
+--------------------+--------------+



In [7]:
from pyspark.sql.functions import when, avg

# Creating the new "energy_level" column
df_transformed = df.withColumn(
    "energy_level",
    when(df["energy"] > 0.8, "High Energy").otherwise("Regular Energy")
)

# Grouping the data by the new energy level classification and calculate the avg popularity and loudness base on this classification
grouped_stats = df_transformed.groupBy("energy_level").agg(
    avg("popularity").alias("avg_popularity"),
    avg("loudness").alias("avg_loudness")
)

# Display the results
print("Energy Level Classification and Aggregated Stats:")
grouped_stats.show()


Energy Level Classification and Aggregated Stats:


+--------------+------------------+------------------+
|  energy_level|    avg_popularity|      avg_loudness|
+--------------+------------------+------------------+
|   High Energy|31.786889141167464|-5.121428733599087|
|Regular Energy|33.990498624800956| 2.625479753796737|
+--------------+------------------+------------------+



In [9]:
# Filtering with: "High Energy"
high_energy = df_transformed.filter(df_transformed["energy_level"] == "High Energy")

# Export to csv the High Energy data
output_path = "high_energy_tracks.csv"
high_energy.write.csv(output_path, header=True, mode="overwrite")

print(f"High Energy tracks exported to {output_path}")


High Energy tracks exported to high_energy_tracks.csv
